In [18]:
import numpy as np
import scipy as sp
import numpy.random as random
import vegas

Particle class

In [43]:
class particle:
     def __init__(self, alive, mu, x, weight):
        self.alive = alive
        self.mu = mu
        self.x = x
        self.weight = weight

Monte Carlo function with inputs

$\sigma_s$, $\sigma_a$, $q_0$, $L$, $W$, and # of trials

In [60]:
def mc(sigmaS, sigmaA, q, L, W, trials):
    sigmaT = sigmaS + sigmaA
    tally = np.zeros(2)
    
    for i in range(trials):
        
        #create particle
        x = W*random.random()
        if random.random() < .5:
            mu = -1
        else:
            mu = 1
        p = particle(True, mu, x, 1)
        
        while p.alive:
            #calculate distance to collision
            distance_to_collision = abs(-np.log(random.random())/sigmaT)
            
            #calculate distance to boundary
            if p.mu == -1:
                distance_to_boundary = p.x
            elif p.mu == 1:
                distance_to_boundary = L-p.x
            
            #tally if distance to collision is greater than distance to boundary
            if distance_to_collision > distance_to_boundary:
                if p.mu == -1:
                    tally[0] += p.weight
                    p.alive = False
                elif p.mu == 1:
                    tally[1] += p.weight
                    p.alive = False
            else:
                p.x += p.mu*distance_to_collision
                if random.random() < sigmaA/sigmaT: #absorbed
                    p.alive = False
                elif random.random() < 0.5: #backscatter
                        p.mu *= -1
                        
    # return flux                
    return tally * q

In [116]:
sigmaS = 1
sigmaA = 1
q = 1
L = 5
W = 2
trials = 1000

mc(sigmaS, sigmaA, q, L, W, trials)

array([176.,   4.])